---
#### 작업형 1-1. 결측치가 있는 데이터의 행을 제거하고, 앞에서부터 70%의 데이터만 이용해 'f1'컬럼 1사분위 값을 구하라 (소수점 제거 후 정수출력)
---

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/members.csv")
print(df.shape)
df.head()

(100, 10)


,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0


In [2]:
df_na = df.dropna()
df_new = df_na[:int(len(df_na)*0.7)]
print('정담 = ', int(df_new['f1'].quantile(0.25)))

정담 =  57


---
#### 작업형 1-2. 2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수를 구하라 (index:연도)
---

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/year.csv', index_col='Unnamed: 0')
print(df.shape)
df.head()

(3, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1999,137,74,114,140,80,150,16,133,178,181,...,124,94,118,12,50,191,137,174,56,128
2000,176,87,64,110,128,16,8,4,123,87,...,59,22,3,108,17,104,101,161,156,43
2001,128,132,123,112,134,160,180,47,115,155,...,160,111,116,75,48,51,81,101,194,73


In [4]:
df_t = df.T
df_new = df_t.iloc[:,1]
print('정답 = ', (df_new > df_new.mean()).sum())

정답 =  100


---
#### 작업형 1-3. 결측치가 가장 많은 컬럼명을 출력하라
---

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/members.csv")
print(df.shape)
df.head()

(100, 10)


,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0


In [6]:
temp = df.isnull().sum()
print('정답 = ', temp.sort_values(ascending=False).index[0])

정답 =  f1


---
#### 작업형 2-1. 여행보험 패키지 상품을 구매할 확률값을 예측하라
- 타겟 : TravelInsurance (패키지 구매여부, 0:구매안함 / 1:구매함)
- 평가 : ROC AUC
- 제출 : result.csv (index, y_pred)
---

In [45]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/test.csv")
print(train.shape, test.shape)
display(train.head(2), test.head(2))

################################## EDA ######################################
# 수치형, 범주형 변수 혼재. 결측치는 없음
# train/test간 obj변수들의 카테고리 종류는 동일함
#############################################################################

# 1. 타겟분리, id제거
y = train['TravelInsurance']
train = train.drop(['TravelInsurance', 'Unnamed: 0'], axis=1)

test_id = test['Unnamed: 0']
test = test.drop('Unnamed: 0', axis=1)

# 수치형변수에 스케일링 추가
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = RobustScaler()
num_cols = train.select_dtypes(exclude='object').columns
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

# 2. 원핫 인코딩
#train = pd.get_dummies(train)
#test = pd.get_dummies(test)

# 2. LabelEncoding
obj_cols = train.select_dtypes('object').columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in obj_cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

# 3. 검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)

# 4. model, eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=120, n_estimators=300)
model.fit(X_train, y_train)
model_pred = model.predict_proba(X_valid)
score = roc_auc_score(y_valid, model_pred[:,1])
print('ROC AUC = ', score)

(1490, 10) (497, 9)


,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0


,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,920,32,Government Sector,Yes,650000,5,0,No,No
1,777,27,Government Sector,Yes,500000,5,0,No,No


ROC AUC =  0.8079877112135176


In [15]:
#submit score
# baseline score    = 0.802899385560676
# Standard scaling  = 0.8031874039938556
# MinMax scaling    = 0.8022273425499231
# Robust scaling    = 0.8023713517665132
# LabelEncoder + RobustSccaler + n_estimators = 300 = 0.8079877112135176

pred = model.predict_proba(test)
result = pd.DataFrame({
    'index':test_id, 'y_pred':pred[:,1]
})
result.to_csv('result.csv', index=False)

In [39]:
##### 참고: Cross Validation
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch3/test.csv")
print(train.shape, test.shape)
display(train.head(2), test.head(2))

################################## EDA ######################################
# 수치형, 범주형 변수 혼재. 결측치는 없음
# train/test간 obj변수들의 카테고리 종류는 동일함
#############################################################################

# 1. 타겟분리, id제거
y = train['TravelInsurance']
train = train.drop(['TravelInsurance', 'Unnamed: 0'], axis=1)

test_id = test['Unnamed: 0']
test = test.drop('Unnamed: 0', axis=1)

# 수치형변수에 스케일링 추가
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = RobustScaler()
num_cols = train.select_dtypes(exclude='object').columns
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

# 2. LabelEncoding
obj_cols = train.select_dtypes('object').columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in obj_cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

# 3. model, eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=120)
cv_scores = cross_val_score(model, train, y, cv=3, scoring='roc_auc')

print('avg ROC AUC = ', cv_scores.mean())

(1490, 10) (497, 9)


,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0


,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,920,32,Government Sector,Yes,650000,5,0,No,No
1,777,27,Government Sector,Yes,500000,5,0,No,No


avg ROC AUC =  0.798724418387157
